# Helplers

## Tree class

In [235]:
class Directory:
    def __init__(self, name, depth=0):
        self.name = name
        self.sub_directories: list[Directory] = []
        self.total_size = 0
        self.file_size = 0
        self.parent: Directory | None = None
        self.files: list[File] = []
        self.depth = depth
     
    def check_is_already_sub_directory(self, name_sub_directory):
        return len([sub_d for sub_d in self.sub_directories if sub_d.name == name_sub_directory]) != 0
    
    def check_is_file_already_added(self, file_name):
        return len([file for file in self.files if file.name == file_name]) != 0    

    def add_sub_directory(self, name_sub_directory):
        if self.check_is_already_sub_directory(name_sub_directory):
            return
        new_sub_directory = Directory(name=name_sub_directory, depth=self.depth + 1)
        new_sub_directory.parent = self
        self.sub_directories.append(new_sub_directory)
    
    def get_sub_directory_instance(self, name_sub_directory):
        for sub_directory in self.sub_directories:
            if name_sub_directory == sub_directory.name:
                return sub_directory
    
    def adding_file_into_dir(self, file_name, file_size):
        if self.check_is_file_already_added(file_name):
            return
        file = File(file_name, file_size)
        self.file_size += int(file_size)
        self.files.append(file)
        
    def set_folder_size(self):
        total_size = self.file_size
        for sub_folder in self.sub_directories:
            total_size += sub_folder.total_size
        self.total_size = total_size

            
        
    
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = int(size)

## Helpers functions

In [236]:
def check_is_file(args):
    return args[0] != "$"

def check_is_dir(args):
    return args[0] == "dir"

def command_is_cd(args):
    return args[1] == "cd"

def command_is_ls(args):
    return args[1] == "ls"

def init_tree():
    return Directory(name="/", depth=0)

def create_folder_tree(lines):
    current_dir = init_tree()
    root_folder = current_dir
    for line in lines[1:]:
        args = line.strip().split(" ")
        if command_is_ls(args):
            continue
        elif check_is_dir(args):
            current_dir.add_sub_directory(args[1])
        elif check_is_file(args):
            current_dir.adding_file_into_dir(args[1], args[0])
        elif command_is_cd(args):
            # Logic to take aways
            if args[2] == "..":
                current_dir = current_dir.parent
            else:
                current_dir = current_dir.get_sub_directory_instance(args[2])
    return root_folder

def display_folder_tree(folder):
    tab_size = folder.depth * "    "
    if folder.parent == None:
        print(f"{tab_size}- DIR {folder.name} (SIZE: {folder.total_size})")
    for sub_folder in folder.sub_directories:
        print(f"{tab_size}- DIR {sub_folder.name} (SIZE: {sub_folder.total_size})")
        display_folder_tree(sub_folder)
    for file in folder.files:
        print(f"{tab_size}- FILE {file.name} (SIZE: {file.size})")
    return

def set_all_folders_size(folder):
    if not folder.sub_directories:
        folder.total_size = folder.file_size
    for sub_folder in folder.sub_directories:
        set_all_folders_size(sub_folder)
    folder.set_folder_size()
    
def get_sum_of_all_file_less_than_specific_size(folder, specific_size=100000, result_array=None):
    if result_array is None:
        result_array=[]
    if (size_to_add := folder.total_size) < specific_size:
        result_array.append(size_to_add)
    for sub_folder in folder.sub_directories:
        get_sum_of_all_file_less_than_specific_size(sub_folder,
                                                    specific_size=specific_size,
                                                    result_array=result_array)
    return sum(result_array)

def get_all_file_more_than_specific_size(folder, specific_size=100000, result_array=None):
    if result_array is None:
        result_array=[]
    if (size_to_add := folder.total_size) >= specific_size:
        result_array.append(size_to_add)
    for sub_folder in folder.sub_directories:
        get_all_file_more_than_specific_size(sub_folder,
                                                    specific_size=specific_size,
                                                    result_array=result_array)
    return result_array

# Problem 1

In [237]:
with open("data/day_7.txt", "r") as file:
    lines = file.readlines()
    root_folder = create_folder_tree(lines)
    set_all_folders_size(root_folder)
    #display_folder_tree(root_folder)
    result = get_sum_of_all_file_less_than_specific_size(root_folder)
    print(result)

1084134


# Problem 2

In [238]:
with open("data/day_7.txt") as file:
    lines = file.readlines()
    root_folder = create_folder_tree(lines)
    set_all_folders_size(root_folder)
    #display_folder_tree(root_folder)
    size_of_system = root_folder.total_size
    size_to_free = 30000000 - (70000000 - size_of_system)
    print(min(get_all_file_more_than_specific_size(root_folder, specific_size=size_to_free)))

6183184
